In [1]:
import os
import speech_recognition as sr
from gtts import gTTS
from playsound import playsound
from langchain_groq import ChatGroq
from dotenv import load_dotenv
import time
import threading

In [2]:
# Load environment variables
load_dotenv()

# Initialize components
recognizer = sr.Recognizer()
microphone = sr.Microphone()

In [3]:
# Initialize ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="mixtral-8x7b-32768"
)

In [4]:
SYSTEM_PROMPT = """
You are a friendly, patient language learning assistant. Help users learn their target language through conversation.
Follow these rules:
1. Use simple, clear explanations
2. Correct mistakes gently with examples
3. Focus on practical conversation
4. Explain grammar when needed
5. Keep responses concise
6. Use voice-friendly formatting
7. Provide cultural context
8. Offer encouragement
9. Stop immediately if user says "stop"
Current teaching language: {language}
"""

In [5]:
# Global flag for interruption
stop_flag = False

def text_to_speech(text):
    """Convert text to speech with interrupt capability"""
    global stop_flag
    try:
        tts = gTTS(text=text, lang='en', slow=False)
        tts.save("response.mp3")
        
        # Play audio in thread
        def play_audio():
            global stop_flag
            if not stop_flag:
                playsound("response.mp3")
            os.remove("response.mp3")
            
        audio_thread = threading.Thread(target=play_audio)
        audio_thread.start()
        
        # Check for stop during playback
        while audio_thread.is_alive():
            if stop_flag:
                return False
            time.sleep(0.1)
            
        return True
        
    except Exception as e:
        print(f"TTS Error: {e}")
        return False

def get_voice_input():
    """Get voice input with stop detection"""
    global stop_flag
    with microphone as source:
        recognizer.adjust_for_ambient_noise(source)
        print("Speak now...")
        try:
            audio = recognizer.listen(source, timeout=5)
            text = recognizer.recognize_google(audio).lower()
            
            if "stop" in text:
                stop_flag = True
                return "stop"
                
            return text
            
        except sr.UnknownValueError:
            return ""
        except Exception as e:
            print(f"Recognition Error: {e}")
            return ""

def generate_response(user_input, language):
    """Generate AI response with stop awareness"""
    global stop_flag
    if stop_flag:
        return ""
        
    prompt = SYSTEM_PROMPT.format(language=language)
    
    try:
        response = llm.invoke([{
            "role": "system", 
            "content": prompt
        }, {
            "role": "user",
            "content": user_input + " (respond in {language})"
        }])
        
        return response.content
        
    except Exception as e:
        print(f"API Error: {e}")
        return ""

In [ ]:
def main():
    global stop_flag
    target_language = input("Enter target language: ").strip()
    
    print(f"Learning {target_language} - say 'stop' to exit")
    text_to_speech(f"Let's learn {target_language}. Begin speaking.")
    
    while not stop_flag:
        user_input = get_voice_input()
        
        if stop_flag:
            break
            
        if not user_input:
            continue
            
        # Generate and process response
        response = generate_response(user_input, target_language)
        
        if response and not stop_flag:
            print(f"Teacher: {response}")
            if not text_to_speech(response):
                break
                
        time.sleep(0.5)

    text_to_speech("Session ended. Goodbye!")

: 

In [ ]:
if __name__ == "__main__":
    main()

Learning English - say 'stop' to exit
Speak now...
Teacher: Hello Rahul, nice to meet you! In English, we would say: "Hello, my name is Rahul and I am a B.Tech student." It's great to be learning English! Is there a specific topic or question you would like to practice in English today?



    Error 263 for command:
        open response.mp3
    The specified device is not open or is not recognized by MCI.

    Error 263 for command:
        close response.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: response.mp3
Exception in thread Thread-5 (play_audio):
Traceback (most recent call last):
  File "E:\anaconda\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "d:\Rahul-Github\Daily-Task\venv\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "E:\anaconda\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\rahul\AppData\Local\Temp\ipykernel_7364\461104492.py", line 15, in play_audio
  File "d:\Rahul-Github\Daily-Task\venv\Lib\site-packages\playsound.py", line 72, in _playsoundWin
    winCommand(u'open {}'.format(sound))
  File "d:\Rahul-Github\Daily-Task\venv\Lib\site-packages\playsound.py", line 64

Speak now...
Speak now...
Speak now...
Speak now...
